In [143]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [144]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)


Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [145]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)


Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [146]:
# first of all. define the accuracy function
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [38]:
# FIRST SITUATION!#

# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
# WITHOUT the batch_size and the hidden_layer, just use the whole data when every time optimized the loss
# and the l2 regularization
train_subset = 10000
beta=0.005
graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+beta*tf.nn.l2_loss(weights)
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.6).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [39]:
# focus!! if you find you can not use the global_variables_initializer, it is because you do not use the same version 
# as the author use, you can upgrade the tensorflow or change the old one to 'initialize_all_variables', it can also 
# work well!
num_steps = 2401

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))


Initialized
Loss at step 0: 36.081543
Training accuracy: 5.0%
Validation accuracy: 6.3%
Loss at step 100: 8.739820
Training accuracy: 73.6%
Validation accuracy: 72.4%
Loss at step 200: 5.236393
Training accuracy: 76.9%
Validation accuracy: 75.3%
Loss at step 300: 3.276453
Training accuracy: 80.0%
Validation accuracy: 77.0%
Loss at step 400: 2.155909
Training accuracy: 82.0%
Validation accuracy: 78.5%
Loss at step 500: 1.511225
Training accuracy: 83.3%
Validation accuracy: 80.0%
Loss at step 600: 1.138133
Training accuracy: 84.4%
Validation accuracy: 80.9%
Loss at step 700: 0.920671
Training accuracy: 85.0%
Validation accuracy: 81.6%
Loss at step 800: 0.793022
Training accuracy: 85.4%
Validation accuracy: 82.1%
Loss at step 900: 0.717633
Training accuracy: 85.6%
Validation accuracy: 82.2%
Loss at step 1000: 0.672882
Training accuracy: 85.7%
Validation accuracy: 82.3%
Loss at step 1100: 0.646206
Training accuracy: 85.8%
Validation accuracy: 82.4%
Loss at step 1200: 0.630248
Training accu

In [ ]:
## in the first SITUATION, i got the best accuracy is 89.0%,(changing the step_number, learning rate, beta) 

In [111]:
# SECOND SITUATION: ##

# using the l2 regularization, and the SGD, but without the hidden layer
batch_size = 240
beta=0.001
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+beta*tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.3).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)


In [112]:
# change the same thing! 'global_variables_initgializer'

num_steps = 9501

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 20.895992
Minibatch accuracy: 5.0%
Validation accuracy: 9.5%
Minibatch loss at step 500: 2.731956
Minibatch accuracy: 85.0%
Validation accuracy: 74.5%
Minibatch loss at step 1000: 1.777867
Minibatch accuracy: 85.0%
Validation accuracy: 76.3%
Minibatch loss at step 1500: 1.913505
Minibatch accuracy: 75.0%
Validation accuracy: 76.4%
Minibatch loss at step 2000: 1.244706
Minibatch accuracy: 75.0%
Validation accuracy: 77.7%
Minibatch loss at step 2500: 2.083354
Minibatch accuracy: 70.0%
Validation accuracy: 76.8%
Minibatch loss at step 3000: 1.066975
Minibatch accuracy: 85.0%
Validation accuracy: 78.1%
Minibatch loss at step 3500: 1.057194
Minibatch accuracy: 70.0%
Validation accuracy: 78.8%
Minibatch loss at step 4000: 0.511023
Minibatch accuracy: 90.0%
Validation accuracy: 78.7%
Minibatch loss at step 4500: 0.892979
Minibatch accuracy: 80.0%
Validation accuracy: 76.7%
Minibatch loss at step 5000: 0.784841
Minibatch accuracy: 80.0%
Validation accuracy

In [ ]:
# in the SECOND situation, i got the best accuracy is 90.0%, (changing the step numbers, learning rate and beta)

In [113]:
# THIRD SITUATION:##
# using the l2 Regularization, but this time i add a hidden layer into the network. and also using the SGD function
# add a hidden layer with 1024 nodes, the input data is the same as the network without hidden  layer.
batch_size = 128
hidden_layer = 1024
beta = 0.001
beta2 = 0.001
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables. here!! we have two weights array here, the first is the input to the hidden , the 
  # the second the the hidden layer to the output layer, and also the biases.
  weights_input_hidden = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer]))
  biases_hidden = tf.Variable(tf.zeros([hidden_layer]))
  weights_hidden_output = tf.Variable(
    tf.truncated_normal([hidden_layer, num_labels]))
  biases_output = tf.Variable(tf.zeros([num_labels]))

  # Training computation. !!notice the hidden layer, using the nn.relu() function to do the math
  # the hidden layer output
  logits_hidden = tf.matmul(tf_train_dataset, weights_input_hidden) + biases_hidden
  # then the output layer output
  logits_output = tf.matmul(tf.nn.relu(logits_hidden), weights_hidden_output) + biases_output
  # finally, we computate the loss of the whole net
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits_output, tf_train_labels))+(beta*tf.nn.l2_loss(weights_input_hidden)
            + beta2*tf.nn.l2_loss(weights_hidden_output))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.4).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_output)
  hidden_logits_valid = tf.matmul(tf_valid_dataset, weights_input_hidden) + biases_hidden
  output_logits_valid = tf.matmul(tf.nn.relu(hidden_logits_valid), weights_hidden_output) + biases_output
  valid_prediction = tf.nn.softmax(output_logits_valid)
  hidden_logits_test = tf.matmul(tf_test_dataset, weights_input_hidden) + biases_hidden
  output_logits_test = tf.matmul(tf.nn.relu(hidden_logits_test), weights_hidden_output) + biases_output
  test_prediction = tf.nn.softmax(output_logits_test)


In [114]:
num_steps = 12001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 660.260010
Minibatch accuracy: 0.0%
Validation accuracy: 17.2%
Minibatch loss at step 500: 284.837311
Minibatch accuracy: 66.7%
Validation accuracy: 60.3%
Minibatch loss at step 1000: 208.061981
Minibatch accuracy: 58.3%
Validation accuracy: 62.7%
Minibatch loss at step 1500: 141.018463
Minibatch accuracy: 50.0%
Validation accuracy: 65.9%
Minibatch loss at step 2000: 79.403130
Minibatch accuracy: 75.0%
Validation accuracy: 65.8%
Minibatch loss at step 2500: 53.339161
Minibatch accuracy: 66.7%
Validation accuracy: 75.0%
Minibatch loss at step 3000: 34.852158
Minibatch accuracy: 83.3%
Validation accuracy: 77.4%
Minibatch loss at step 3500: 23.500515
Minibatch accuracy: 91.7%
Validation accuracy: 79.8%
Minibatch loss at step 4000: 16.107725
Minibatch accuracy: 83.3%
Validation accuracy: 78.1%
Minibatch loss at step 4500: 10.780450
Minibatch accuracy: 83.3%
Validation accuracy: 78.9%
Minibatch loss at step 5000: 7.390801
Minibatch accuracy: 91.7%
Valid

In [ ]:
# in the THIRD SITUATION , i got the best accuracy is 94.8, 
# at first , i just add the l2_loss of the weights between hidden layer and output layer, and the accuracy i got is just 
# 91%, but when i just add the l2_loss of the weights between input and hidden layer, i got 94%!, and so i add two l2_loss
# of both weights array, and the accuracy is better. The l2_loss of weights between input and hidden is more important!!

In [ ]:
##  NEXT!  i use the dropout function, which can be used when the data is not enough or not big, still remember the hidden
# layer is still used.  ## DROPOUT ##

In [153]:
# do some experiment using the dropout, remember using the dropout in the training_data, 'nn.dropout()'
batch_size = 128
hidden_layer = 1024
beta = 0.001
beta2 = 0.001
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables. here!! we have two weights array here, the first is the input to the hidden , the 
  # the second the the hidden layer to the output layer, and also the biases.
  weights_input_hidden = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer]))
  biases_hidden = tf.Variable(tf.zeros([hidden_layer]))
  weights_hidden_output = tf.Variable(
    tf.truncated_normal([hidden_layer, num_labels]))
  biases_output = tf.Variable(tf.zeros([num_labels]))

  # Training computation. !!notice the hidden layer, using the nn.relu() function to do the math
  # the hidden layer output
  logits_hidden = tf.matmul(tf_train_dataset, weights_input_hidden) + biases_hidden
  # then the output layer output
  logits_output = tf.matmul(tf.nn.dropout(logits_hidden,0.4), weights_hidden_output) + biases_output
  # finally, we computate the loss of the whole net
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits_output, tf_train_labels))+(beta*tf.nn.l2_loss(weights_input_hidden)
            + beta2*tf.nn.l2_loss(weights_hidden_output))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.4).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_output)
  hidden_logits_valid = tf.matmul(tf_valid_dataset, weights_input_hidden) + biases_hidden
  output_logits_valid = tf.matmul(hidden_logits_valid, weights_hidden_output) + biases_output
  valid_prediction = tf.nn.softmax(output_logits_valid)
  hidden_logits_test = tf.matmul(tf_test_dataset, weights_input_hidden) + biases_hidden
  output_logits_test = tf.matmul(hidden_logits_test, weights_hidden_output) + biases_output
  test_prediction = tf.nn.softmax(output_logits_test)


In [154]:
num_steps = 12001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 965.924988
Minibatch accuracy: 17.0%
Validation accuracy: 39.8%
Minibatch loss at step 500: 1095.283203
Minibatch accuracy: 64.0%
Validation accuracy: 72.6%


KeyboardInterrupt: 

In [ ]:
## in the dropout experiment, when i use the nn.drop() in the hidden layer, can not get a better accuracy, expecially when
# i choose a few batch size, the dropout seems can't help at all！！

In [179]:
# do some experiment using l2_loss,##no dropout## and multiple layers 
batch_size = 256
beta1=0.001
beta2=0.005
beta3=0.005
#larning_rate=0.001
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Network Parameters
  n_hidden_1 = 1024  # 1st layer number of features
  n_hidden_2 = 256  # 2nd layer number of features
  n_input = 784  # notMNIST data input (img shape: 28*28)
  n_classes = 10  # notMNIST total classes (0-9 labels)

  # Store layers weight & bias
  weights = {
    'h1': tf.Variable(tf.truncated_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.truncated_normal([n_hidden_2, n_classes]))
  }
  biases = {
    'b1': tf.Variable(tf.zeros([n_hidden_1])),
    'b2': tf.Variable(tf.zeros([n_hidden_2])),
    'out': tf.Variable(tf.zeros([n_classes]))
  }

  
  # Variables.
  #weights = tf.Variable(
  # tf.truncated_normal([image_size * image_size, num_labels]))
  #biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.--------
  # using multiple layers!
  # the first layer to the first hidden layer
  layer_1 = tf.matmul(tf_train_dataset, weights['h1']) + biases['b1']
  layer_1 = tf.nn.relu(layer_1)
  # the first hidden layer to the second hidden layer
  layer_2 = tf.matmul(layer_1, weights['h2']) + biases['b2']
  layer_2 = tf.nn.relu(layer_2)
  # the output layer
  logits = tf.matmul(layer_2, weights['out']) + biases['out']
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+(beta1*tf.nn.l2_loss(weights['out'])
            + beta2*tf.nn.l2_loss(weights['h2']) + beta3*tf.nn.l2_loss(weights['h1']))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  # validation
  layer_1_valid=tf.matmul(tf_valid_dataset, weights['h1']) + biases['b1']
  layer_1_valid = tf.nn.relu(layer_1_valid)
  layer_2_valid=tf.matmul(layer_1_valid, weights['h2']) + biases['b2']
  layer_2_valid = tf.nn.relu(layer_2_valid)
  outlayer_valid=tf.matmul(layer_2_valid, weights['out']) + biases['out']
  # test_data
  layer_1_test=tf.matmul(tf_test_dataset, weights['h1']) + biases['b1']
  layer_1_test = tf.nn.relu(layer_1_test)
  layer_2_test=tf.matmul(layer_1_test, weights['h2']) + biases['b2']
  layer_2_test = tf.nn.relu(layer_2_test)
  outlayer_test=tf.matmul(layer_2_test, weights['out']) + biases['out']
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(outlayer_valid)
  test_prediction = tf.nn.softmax(outlayer_test)


In [180]:
num_steps = 40001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 5485.990234
Minibatch accuracy: 11.3%
Validation accuracy: 23.5%
Minibatch loss at step 500: 1665.904297
Minibatch accuracy: 24.2%
Validation accuracy: 23.3%
Minibatch loss at step 1000: 1299.910522
Minibatch accuracy: 30.9%
Validation accuracy: 31.5%
Minibatch loss at step 1500: 1014.923950
Minibatch accuracy: 32.8%
Validation accuracy: 37.4%
Minibatch loss at step 2000: 792.511292
Minibatch accuracy: 43.4%
Validation accuracy: 43.2%
Minibatch loss at step 2500: 619.258972
Minibatch accuracy: 52.7%
Validation accuracy: 52.5%
Minibatch loss at step 3000: 484.196442
Minibatch accuracy: 59.4%
Validation accuracy: 60.9%
Minibatch loss at step 3500: 378.944458
Minibatch accuracy: 66.4%
Validation accuracy: 67.9%
Minibatch loss at step 4000: 296.906128
Minibatch accuracy: 72.7%
Validation accuracy: 71.3%
Minibatch loss at step 4500: 232.935944
Minibatch accuracy: 71.1%
Validation accuracy: 72.7%
Minibatch loss at step 5000: 183.318283
Minibatch accuracy

In [ ]:
# enlarge the beat2 and beta3 can take less time or step number to get a lower loss!! got best accuracy is 95%
# but i notice that , when i choose beta2=beta3=0.05 , i got the loss=50 at  8000 setp number, but when i choose 
# the beta2=beta3=0.01, in order to get the same loss=50, i got it almost at setp number =30000. it is a big improvement!

# BUT there is still a qusetion remained, cause i am not sure how to use the dropout in the network, cause when using the 
# dropout, i can not get better acccuracy, even i got a worse accuracy! Disiappointed!